In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics
from sklearn import tree
from sklearn.preprocessing import scale
from sklearn.metrics import classification_report, accuracy_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
#from sklearn.cluster import AgglomerativeClustering
#from sklearn.cluster import Birch
#from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import MeanShift
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from tensorflow import keras
from keras.api._v2.keras import activations

In [4]:
anemia_df=pd.read_csv('CBC-data_for_meandeley_csv.csv')

In [ ]:
anemia_df

,S. No.,Age,Sex,RBC,PCV,MCV,MCH,MCHC,RDW,TLC,PLT /mm3,HGB
0,NaN,NaN,NaN,Red Blood Cell count,Packed Cell Volume,Mean Cell Volume,Mean Cell Hemoglobin,NaN,Red Cell Distribution width,"White Blood Cell (WBC count),",Platelet,Hemoglobin
1,1.0,28.0,0.0,5.66,34,60.1,17,28.2,20,11.1,128.3,9.6
2,2.0,41.0,0.0,4.78,44.5,93.1,28.9,31.0,13,7.02,419,13.8
3,3.0,40.0,1.0,4.65,41.6,89.5,28.8,32.2,13,8.09,325,13.4
4,4.0,76.0,0.0,4.24,36.7,86.6,26.7,30.8,14.9,13.41,264,11.3
...,...,...,...,...,...,...,...,...,...,...,...,...
369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
anemia_df.describe()

,S. No.,Age,Sex,MCHC
count,364.000000,364.000000,364.000000,364.000000
mean,182.500000,44.917582,0.442308,32.053407
std,105.221988,18.780854,0.497344,2.801790
min,1.000000,11.000000,0.000000,23.600000
25%,91.750000,28.000000,0.000000,30.300000
50%,182.500000,44.500000,0.000000,31.700000
75%,273.250000,60.000000,1.000000,33.300000
max,364.000000,89.000000,1.000000,50.200000


In [6]:
anemia_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   S. No.     364 non-null    float64
 1   Age        364 non-null    float64
 2   Sex        364 non-null    float64
 3     RBC      365 non-null    object 
 4   PCV        365 non-null    object 
 5   MCV        365 non-null    object 
 6   MCH        365 non-null    object 
 7    MCHC      364 non-null    float64
 8    RDW       365 non-null    object 
 9   TLC        365 non-null    object 
 10   PLT /mm3  365 non-null    object 
 11   HGB       365 non-null    object 
dtypes: float64(4), object(8)
memory usage: 35.2+ KB


In [7]:
correlation_mat = anemia_df.corr()   #correlation matrix between columns of dataframe before preprocessing
correlation_mat

<ipython-input-7-f8747ed81e51>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_mat = anemia_df.corr()   #correlation matrix between columns of dataframe before preprocessing


,S. No.,Age,Sex,MCHC
S. No.,1.000000,0.051263,0.120786,0.204934
Age,0.051263,1.000000,-0.096953,0.092982
Sex,0.120786,-0.096953,1.000000,-0.200581
MCHC,0.204934,0.092982,-0.200581,1.000000


In [8]:
anemia_df.duplicated().sum().sum()

8

In [9]:
anemia_df.isnull().sum().sum()

112

In [10]:
anemia_df= anemia_df.drop_duplicates() #drop duplicated rows '8rows'

In [11]:
anemia_df.duplicated().sum().sum()

0

In [12]:
anemia_df=anemia_df.dropna(how='all') #drop all nan rows
anemia_df=anemia_df.dropna(how='any')  #drop nan rows that contain at least one null values 'row number 0'
anemia_df=anemia_df.drop(['S. No.'], axis=1)
anemia_df=anemia_df.drop(['Age      '], axis=1)
anemia_df=anemia_df.drop(['Sex  '], axis=1)

In [13]:
anemia_df

,RBC,PCV,MCV,MCH,MCHC,RDW,TLC,PLT /mm3,HGB
1,5.66,34,60.1,17,28.2,20,11.1,128.3,9.6
2,4.78,44.5,93.1,28.9,31.0,13,7.02,419,13.8
3,4.65,41.6,89.5,28.8,32.2,13,8.09,325,13.4
4,4.24,36.7,86.6,26.7,30.8,14.9,13.41,264,11.3
5,4.14,36.9,89.1,27.8,31.2,13.2,4.75,196,11.5
...,...,...,...,...,...,...,...,...,...
360,4.86,39.2,80.7,27.7,34.4,13.4,4.4,180,13.5
361,4.85,35.2,91.7,31,33.8,13.3,5.6,215,15
362,4.47,39.7,88.7,29.3,33.0,13.5,9.2,329,13.1
363,4.75,36.2,86.7,27.9,32.1,13.5,6.48,174,13.2


In [14]:
anemia_df.isnull().sum().sum()

0

In [15]:
kmeans = KMeans(n_clusters=2)
kmeans

KMeans(n_clusters=2)

In [16]:
kmeans.fit(anemia_df)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=2)

In [17]:
kmeans.labels_

array([1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,

In [18]:
#model = AgglomerativeClustering(n_clusters=3)
#yhat = model.fit_predict(anemia_df)
#model = Birch(threshold=0.5, n_clusters=3)
#model.fit(anemia_df)
#yhat = model.predict(anemia_df)
#model = MiniBatchKMeans(n_clusters=3)
#model.fit(anemia_df)
#yhat = model.predict(anemia_df)
#model = MeanShift()
#yhat = model.fit_predict(anemia_df)

In [19]:
anemia_df['cluster']=kmeans.labels_

In [20]:
anemia_df.head()

,RBC,PCV,MCV,MCH,MCHC,RDW,TLC,PLT /mm3,HGB,cluster
1,5.66,34,60.1,17,28.2,20,11.1,128.3,9.6,1
2,4.78,44.5,93.1,28.9,31.0,13,7.02,419,13.8,0
3,4.65,41.6,89.5,28.8,32.2,13,8.09,325,13.4,0
4,4.24,36.7,86.6,26.7,30.8,14.9,13.41,264,11.3,0
5,4.14,36.9,89.1,27.8,31.2,13.2,4.75,196,11.5,1


In [21]:
anemia_df.to_csv('updated_cbc.csv', index=False)

In [22]:
correlation_mat = anemia_df.corr()    #correlation matrix between columns of dataframe after preprocessing
correlation_mat

<ipython-input-22-47ab827ad16e>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_mat = anemia_df.corr()    #correlation matrix between columns of dataframe after preprocessing


,MCHC,cluster
MCHC,1.000000,0.122991
cluster,0.122991,1.000000


In [23]:
Y= anemia_df['cluster']
col = "cluster"
X = anemia_df.loc[:, anemia_df.columns != col] #select all dataframe except cluter column

In [24]:
scaler=MinMaxScaler()       #scale data for mean 0 and unit variance
X=scaler.fit_transform(X)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1,shuffle=True)

In [26]:
X_train

array([[0.61191336, 0.46347032, 0.48976608, ..., 0.15833746, 0.22615385,
        0.58441558],
       [0.59025271, 0.63926941, 0.48538012, ..., 0.19792182, 0.40153846,
        0.59090909],
       [0.63176895, 0.456621  , 0.27777778, ..., 0.10638298, 0.38153846,
        0.44155844],
       ...,
       [0.44765343, 0.45205479, 0.43859649, ..., 0.1410193 , 0.39384615,
        0.38961039],
       [0.60288809, 0.64383562, 0.47076023, ..., 0.19470559, 0.41692308,
        0.56493506],
       [0.46028881, 0.50684932, 0.50584795, ..., 0.17516081, 0.22      ,
        0.46753247]])

In [27]:
X_train.shape

(291, 9)

In [28]:
svm_clf =svm.SVC(kernel='rbf',gamma='auto',C=2)
svm_clf.fit(X_train, y_train)
PREDICTION=svm_clf.predict(X_test)
PREDICTION

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1], dtype=int32)

In [29]:
r=sklearn.metrics.confusion_matrix(y_test, PREDICTION)
r=np.flip(r)

In [30]:
acc1=(r[0][0]+r[-1][-1])/np.sum(r)
acc1

0.8356164383561644

In [31]:
logistic_clf = LogisticRegression()
logistic_clf.fit(X_train, y_train)
y_pred = logistic_clf.predict(X_test)

In [32]:
coef_num= logistic_clf.coef_

In [33]:
coef_num

array([[ 0.34549585, -0.25004632,  0.27980614,  0.19293145,  0.28639608,
         0.05801673, -1.12030755, -6.91619741,  0.34951276]])

In [34]:
intercipt= logistic_clf.intercept_

In [35]:
intercipt

array([2.8402358])

In [36]:
y_pred

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1], dtype=int32)

In [37]:
acc2= accuracy_score(y_test, y_pred)
acc2

0.8356164383561644

In [38]:
kn_clf=KNeighborsClassifier(n_neighbors=1)
kn_clf.fit(X_train,y_train)
y_predicted= kn_clf.predict(X_test)

In [39]:
y_predicted

array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0], dtype=int32)

In [40]:
acc3=accuracy_score(y_test,y_predicted)
acc3

0.9178082191780822

In [41]:
model= keras.Sequential()
hidden_layer_sizes= coef_num.shape[1:]
model.add(keras.layers.Dense(units=hidden_layer_sizes[0],activation='relu',input_shape=(X_train.shape[1],)))
for layer_size in hidden_layer_sizes[1:]:
  model.add(keras.layers.Dense(units=layer_size,activation='relu'))

model.add(keras.layers.Dense(units=coef_num[-1].shape[0],activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 90        
                                                                 
 dense_1 (Dense)             (None, 9)                 90        
                                                                 
Total params: 180
Trainable params: 180
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train , y_train, epochs=500, batch_size=120)

Epoch 1/500
3/3 [==============================] - 1s 8ms/step - loss: 2.4227 - accuracy: 0.0069
Epoch 2/500
3/3 [==============================] - 0s 5ms/step - loss: 2.3883 - accuracy: 0.0069
Epoch 3/500
3/3 [==============================] - 0s 8ms/step - loss: 2.3546 - accuracy: 0.0206
Epoch 4/500
3/3 [==============================] - 0s 5ms/step - loss: 2.3221 - accuracy: 0.0653
Epoch 5/500
3/3 [==============================] - 0s 5ms/step - loss: 2.2904 - accuracy: 0.1443
Epoch 6/500
3/3 [==============================] - 0s 6ms/step - loss: 2.2593 - accuracy: 0.2302
Epoch 7/500
3/3 [==============================] - 0s 5ms/step - loss: 2.2290 - accuracy: 0.2784
Epoch 8/500
3/3 [==============================] - 0s 7ms/step - loss: 2.1993 - accuracy: 0.2955
Epoch 9/500
3/3 [==============================] - 0s 6ms/step - loss: 2.1701 - accuracy: 0.3024
Epoch 10/500
3/3 [==============================] - 0s 5ms/step - loss: 2.1411 - accuracy: 0.3024
Epoch 11/500
3/3 [===========

In [43]:
# input >> 5.66,34,60.1,17,28.2,20,11.1,128.32,9.6
print("Enter Values of CBC test:")
list = [[float(input()) for j in range(9)]]
list = np.array(list)
prediction = model.predict(list)
prediction = prediction.all()
if prediction == 0:
  prediction = 0 #infected
else:
  prediction = 1 #noninfected
print(prediction)

Enter Values of CBC test:
5.66
34
60.1
17
28.2
20
11.1
128.32
9.6
1/1 [==============================] - 0s 82ms/step
0


In [44]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
##converter.experimental_new_converter = True

tflite_model = converter.convert()

with open('model.tflite_cbc', 'wb') as f:
  f.write(tflite_model)